In [3]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import train_test_split

import pickle
import random
from string import punctuation

import gensim
from gensim import corpora

import spacy
from spacy.lang.en import English


import nltk
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
from nltk.stem.wordnet import WordNetLemmatizer

nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/katieta/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/katieta/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [43]:
parser = English()
def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        elif token.orth_.startswith('@'):
            lda_tokens.append('SCREEN_NAME')
#         elif token.orth_.startswith('#'):
#             lda_tokens.append('HASHTAG')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma

def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)

def prepare_text_for_lda(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 4]
    tokens = [token for token in tokens if token not in en_stop]
    tokens = [get_lemma(token) for token in tokens]
    return tokens

def get_text_data(corpus, token_len=20):
    text_data = []
    for i in range(len(corpus)):
        if i % 20000 == 0:
            print("{} / {}".format(i, len(corpus)))
#             if i > 0:
#                 print(text_data[i - 1])
        try:
            tokens = prepare_text_for_lda(corpus[i])
            while len(tokens) < token_len:
                tokens.append('STOP')
                
            if len(tokens) > token_len:
                tokens = tokens[:token_len - 1]
                tokens.append('STOP')
                text_data.append(tokens)
            else:
                text_data.append(tokens)
        except Exception as e:
            print(e)
    print("finished")
    return text_data

# remove common words and tokenize
list1 = ['RT','rt']
stoplist = stopwords.words('english') + list(punctuation) + list1
# print(stoplist)
en_stop = set(nltk.corpus.stopwords.words('english'))
# print(stoplist)

def get_dict_corpus(text_data):
    dictionary = corpora.Dictionary(text_data)
    corpus = [dictionary.doc2idx(text) for text in text_data]
    return dictionary, corpus

In [111]:
dem_tweets = pd.read_csv('../data/tweet_data/dem_tweets_cleaned.csv')
dem_tweets['party'] = [0 for i in range(len(dem_tweets))]
rep_tweets =pd.read_csv('../data/tweet_data/rep_tweets_cleaned.csv')
rep_tweets['party'] = [0 for i in range(len(rep_tweets))]
all_tweets = pd.concat([dem_tweets, rep_tweets])

In [112]:
tweet_idx_col = [i for i in range(len(all_tweets))]
all_tweets['tweet_idx'] = tweet_idx_col
all_tweets = all_tweets[['tweet_idx', 'stripped_text', 'party']]

In [113]:
all_tweets.head()

,tweet_idx,stripped_text,party
0,0,With corruption being one of the biggest conce...,0
1,1,With corruption being one of the biggest conce...,0
2,2,Here is an important notice concerning Christo...,0
3,3,I believe that we need to find real solutions ...,0
4,4,"It's time to turn the page on Alabama's past, ...",0


In [114]:
all_tweets['stripped_text'] = all_tweets['stripped_text'].str.lower()

In [115]:
print(len(all_tweets))
print(len(all_tweets[all_tweets['stripped_text'].str.contains('gun') | all_tweets['stripped_text'].str.contains('gun control')]))
# print(len(all_tweets[~all_tweets['stripped_text'].str.contains('gun')]))


557549
11271


In [388]:
gun_control_keywords = [' gun ', 'gun ', 'gun control', 'guncontrol', 'gun reform', 'gunreformnow', 'nra'
                           ,'2nd amendment',  'assault weapon', '2a', 'firearm', 'tree of life', 'columbine'
                            , 'marchforourlives', 'shotgun', 'second amendment', 'gunsense', 'gunsensenow',
                       'momsdemand', 'gunviolence', 'wearorange', 'gunreform', 'semi-automatic', 'assault weapon'
                       , 'assault-type weapons']

healthcare_keywords = ['healthcare', 'health care', 'medicare', 'medicaid', 'health-care', 'affordable care act', 'aca ']

abortion_keywords = ['prolife', 'pro-life', 'pro life', 'prochoice', 'pro-choice', 'pro choice']

def label_topic(tweet, keywords):
    for kw in keywords:
        if kw in tweet:
            return 1
    return 0
    
all_tweets['gun_control'] = all_tweets['stripped_text'].map(lambda x: label_topic(x, gun_control_keywords))
all_tweets['health_care'] = all_tweets['stripped_text'].map(lambda x: label_topic(x, healthcare_keywords))
all_tweets['abortion'] = all_tweets['stripped_text'].map(lambda x: label_topic(x, healthcare_keywords))


In [389]:
topic = 'health_care'

print(len(all_tweets[all_tweets[topic] == 1]))
topic_tweets = pd.concat([all_tweets[all_tweets[topic] == 1], all_tweets[all_tweets[topic] == 0].sample(len(all_tweets[all_tweets[topic] == 1]))])
print(len(gun_control_tweets))
full_tweets = pd.concat([topic_tweets, all_tweets[~all_tweets['tweet_idx'].isin(topic_tweets['tweet_idx'].unique())]])

print(len(all_tweets))
print(len(full_tweets))
# 

20787
27172
557549
557549


In [372]:
for t in topic_tweets[topic_tweets[topic] == 1].sample(100)['stripped_text'].values:
    print(t, '\n')

1. mental processes have never killed anybody - a physical instrument is required to go from thought to action 2. the @gop refuses to grant mental #healthcare as a right, so unless you're going to fix the mental health system, don't bother bringing it up. #guncontrolnow 

talking to media in columbus about my plan to grow jobs, lower healthcare costs, strengthen all communities, invest in public education, and keep our families safe. #gapol #gagop #gafirst #tcot 

this is not a political issue; it's a moral issue. every man, woman, & child needs quality, affordable health care. my opponent voted at least 19 times to repeal the aca & let insurance companies deny coverage for pre-existing conditions. #ga07 go vote. 

hudsonvalley360: greene county ranked 47th in the state in health outcomes; columbia county ranks 20  we are facing a healthcare crisis in our state. 

#womensrights #healthcare #hr771 

i'm in. a third party (i.e. @lwv) entity could moderate. i think it would be fantastic. 

In [373]:
model_token_data = get_text_data(full_tweets['stripped_text'].values)
dictionary, corpus = get_dict_corpus(model_token_data)
print(len(dictionary))
X = [{w: 1 for w in x} for x in model_token_data]
print(X[0])
dv = DictVectorizer()
X = dv.fit_transform(X)

print(X.shape)
# print(X.todense())
# names = dv.get_feature_names()

0 / 557549
20000 / 557549
40000 / 557549
60000 / 557549
80000 / 557549
100000 / 557549
120000 / 557549
140000 / 557549
160000 / 557549
180000 / 557549
200000 / 557549
220000 / 557549
240000 / 557549
260000 / 557549
280000 / 557549
300000 / 557549
320000 / 557549
340000 / 557549
360000 / 557549
380000 / 557549
400000 / 557549
420000 / 557549
440000 / 557549
460000 / 557549
480000 / 557549
500000 / 557549
520000 / 557549
540000 / 557549
finished
122474
{'maddox': 1, 'want': 1, 'expand': 1, 'medicaid': 1, 'office': 1, 'enough': 1, 'alabama': 1, 'medicareforall': 1, 'economic': 1, 'redevelopment': 1, 'provide': 1, 'catalyst': 1, 'need': 1, 'universalhealthcare': 1, 'people': 1, 'rethinkalabama': 1, 'writemein': 1, 'STOP': 1}
(557549, 122474)


In [375]:
print(len(topic_tweets))
topic_subset = X[:len(topic_tweets)]

X_train, X_test, y_train, y_test = train_test_split(topic_subset, topic_tweets[topic].values[:len(topic_tweets)], test_size=0.1)
clf = LogisticRegression()

# with open('pickles/logistic_regression_model.pkl', 'rb') as f:
#     clf = pickle.load(f)

clf.fit(X_train, y_train)
score = clf.score(X_test, y_test)

with open('pickles/healthcare_logistic_regr.pkl', 'wb') as f:
    pickle.dump(clf, f)

for w, n in sorted(zip(clf.coef_[0], names), key=lambda e:e[0], reverse=True):
    print(w, n)

27172
10.535852881014838 healthcare
7.730010356571205 health
6.185515217942225 healthcarefreedom
5.39731612583368 universalhealthcare
5.213812743110419 healthcareforall
4.5713006220991055 healthcarevoter
3.3053231934365024 protectourhealthcare
3.2468208533935634 healthcareisahumanright
2.7126974989970702 healthcarecrisis
2.6557220070162053 saveourhealthcare
2.618381288680789 singlepayerhealthcare
2.613764547970601 yesforhealthcare
2.304220213514818 healthcare4all
2.195775954714863 affordablehealthcare
2.1695676865144167 betterhealthcare
2.1087537545194555 stepup4healthcare
2.080091823368701 healthcareoverpolitics
1.9599096836756666 affordable
1.8676328499579307 measure101
1.7895031903678946 betterjobs
1.7358239737409524 free2care
1.7304354690304196 healthcareisaright
1.6013700371380415 nurse
1.5412447516068704 doctor
1.5149249870768182 betterwages
1.493650145668409 embrace
1.4254135866407958 healthcare-
1.4182170719736689 medicareforall
1.384659121207637 hr676
1.3226123735681938 univer

0.36719832768769156 -immigration
0.36719832768769156 trillion$
0.3669485699346111 sober
0.36673917425760355 abuser
0.36626080659913945 possible
0.36625397063642934 interfere
0.3649993638059029 kirsten
0.3645002178247875 feeling
0.36280649114720764 keeping
0.3626879937935868 lack
0.3623762048511185 obstruction
0.36236234635266007 solution
0.36139010710450864 simply
0.3608144502075977 talking
0.3604037133817146 worker
0.3602074612759038 50/50
0.36020597749400923 brushing
0.36020597749400923 scandinavian
0.3597738036885177 chair
0.35896198990241707 island
0.35880286537235445 money
0.3587437344439337 pilot
0.3585526364522568 turn31blue
0.3581077267083069 greece
0.3580037914909677 nationaldoctorsday
0.35792828135667276 steel
0.35751455567934554 irondequoit
0.3573453083250353 flooding
0.35733099569953797 apprenticeship
0.357199440064196 frederick
0.3571339258228931 basics
0.35662202452318115 mentally
0.35658538542695867 solutuon
0.35652751289367884 ny25s
0.35645719917524654 military
0.356253

0.17240824148849548 educator
0.17214847717147602 spending
0.17213393757563508 compassion
0.17110511245978952 superintendent
0.17069908671251602 primary
0.17059149873267349 oppose
0.17044380536238393 unique
0.17013435346511552 federation
0.17006728420302578 emilys
0.1697918448753023 shout
0.16917407989333566 consciousness
0.16878063718107045 incumbent
0.168398299722523 infection
0.16797679612460156 womenpower
0.16792077053211432 opopportunity
0.16774766143895092 unionize
0.16741141979420368 knowledge
0.16726737880673012 reporter
0.167097881787987 txgovdebate
0.1667096493145549 holding
0.16650734231875175 lifeline
0.16643547401621317 culberson
0.16637344127133855 solvent
0.16611617763338687 diligently
0.16580742866679257 problem
0.16509714523164357 solving
0.16499045095345083 everybody
0.1646858327297779 every
0.16468020524940086 speaker
0.16466639978207337 anthony
0.16439164233340775 in-02
0.16439164233340775 joshi
0.16382402668772844 thrill
0.1637656811617521 righttohealth
0.1636891189

0.053273963316620265 livelihood
0.05323149372532844 taxbreak
0.0532300699070431 kitchen
0.05321534007409835 valentine
0.05307171890049762 nationalhealthcenterweek
0.05285162964546366 stopkillingus
0.05285162964546366 whiteliberallies
0.052642060455761114 arrange
0.05260422222650176 everyonevotes
0.05260422222650176 sf2117/sf2417
0.05252396324992881 govtshutdown
0.05233418907440884 harm
0.052107978148154925 wvian
0.05179878019325056 progressive
0.051753992279802666 741741
0.051753992279802666 spade
0.0516369809836967 preside
0.051627566750783754 precious
0.05152955539133662 actuary
0.051473209062214385 wraparound
0.05125761235829684 huckaby
0.05122588426498352 ashley
0.05121450313168564 fulfill
0.05118498401601301 bexarcounty
0.05105610073817856 undiagnosed
0.0510505613905325 viagra
0.05097335955570705 moderate
0.05093021712015126 il13debate
0.050865983693402506 silently
0.05072945769356273 upton
0.05062417478184958 skimp
0.05060372773317024 charleston
0.05056763889947818 description
0.

0.023908234400616825 frarey
0.023908234400616825 paywall
0.023896237343408063 musician
0.023847266552507095 retail
0.02376775705302594 distract
0.023766525012868793 or2debate
0.023718137347952712 policymaker
0.0236737414645841 disorder
0.023660353775627065 whataboutus
0.023624559788944916 hbcus
0.023589881784691232 inept
0.023547977631363074 merge
0.023527978437038854 congressdoyourjob
0.023489960841554983 500000jobs
0.02348323012989619 blazing
0.02347733896213672 edwforum
0.023435039497067434 louise
0.02336261394650349 fearlessly
0.02332573122832249 underhanded
0.023315885067572973 makehealthcareuniversal
0.023309212884289318 shuffle
0.023294781972984135 forsaking
0.023229341356440916 progressivism
0.023197031063628262 fragile
0.023180450227487076 100k+
0.02317310070192123 hr6137
0.023058728210525837 natiuonwide
0.02303898945481128 winamac
0.022992845126400416 slush
0.022958003131602858 gearing
0.022929139919129593 soaring
0.02292791122979963 may15
0.022879654114927143 endless
0.02286

...

-0.031341286791116675 shoot
-0.031364651612127224 outwork
-0.031364651612127224 summitt
-0.03137979051890709 burnett
-0.03138406570115996 stopthebansc
-0.03138828169877453 instructions
-0.031397893995991785 lady
-0.03140152542083611 persuade
-0.03141467562561228 profiteer
-0.0314248057295516 adams
-0.031425917045524485 sanitation
-0.031436274711106965 amplify
-0.03149199865850812 distort
-0.03149280100089495 institutional
-0.03149477165339356 riley
-0.03149551787889872 invitation
-0.031500158435442474 champ
-0.031506090302999404 execution
-0.0315062952939217 logic
-0.031523109916445545 appropriation
-0.031528516585135336 peterson
-0.03153463671526107 funnel
-0.03155354675820397 mccaskill
-0.03158919669526156 bearable
-0.031591735640692696 repost
-0.031595129624951805 shining
-0.03160116434960982 citizensfirst
-0.031601919422730974 proportion
-0.03160492607585923 turnnmaround
-0.03162153610962984 estate
-0.031625348200792894 coddle
-0.031625348200792894 psychologist
-0.03164460977244673

-0.045713197216382756 judiciary
-0.0457210086784143 humans
-0.04572957134143745 nonetheless
-0.04573101899945455 votered
-0.04576070637408183 teamwhitmer
-0.04580039632465072 plague
-0.045805694854602105 realdeal
-0.045832755793609514 equalpayday
-0.045848640017523846 22-nov
-0.04587698929511048 laccc
-0.04587698929511048 latin
-0.04587984980286312 texasdistrict21
-0.045880434983766774 scandal
-0.045909518801368766 bosma
-0.045909518801368766 impair
-0.045909518801368766 visually
-0.045915421788408226 wanna
-0.04593737955542875 coverage
-0.045941536678632475 oklaed
-0.04595126202301189 cream
-0.045957953227399304 orient
-0.046012129925462074 cabinet
-0.04605230641716745 engineer
-0.04611932558122544 floridian
-0.04612394063552615 assassinate
-0.04621917988557989 writer
-0.04630304076068087 bluewavecoming2018
-0.04632622789235741 shri2018
-0.04634352552422051 leaving
-0.04634918013143035 wexton
-0.04635326802396647 slave
-0.0463692445147792 possibly
-0.04637289225319317 sentence
-0.0463

-0.06797569471414341 fascist
-0.06808465735604652 letsfly
-0.06809360730899107 assassination
-0.06811749397169099 energetic
-0.0681450786208848 theory
-0.068183224801717 kavanaughhearings
-0.06824792237730569 senseless
-0.06830478645565942 dallas
-0.06848988012413353 straw
-0.06854916597299936 meadow
-0.06856062215114901 marginalize
-0.06858714806438927 nobel
-0.06864547233166574 violation
-0.0687022623007255 agent
-0.06885318415776925 austintown
-0.06885318415776925 fitch
-0.06885348074479511 binding
-0.06886035853319382 gubernatorial
-0.06887675182913805 vasen
-0.06889123690591768 grace
-0.06889535299807259 reflection
-0.06892743443546268 impeachment
-0.06897802006469915 teammarsha
-0.06898164996924983 aldems
-0.0690092572539244 dorighttx
-0.06905807860805686 awarding
-0.0690725526895829 violence
-0.06915419664472504 interest
-0.06919533845817334 walkout
-0.06928642335037495 submit
-0.0693794555015526 accused
-0.06943992118275209 latinx
-0.06946304865197626 unanimous
-0.0695059323003

-0.11773995835975773 traditional
-0.11776334953753145 opposite
-0.11776767800083517 norman
-0.11783852565143085 scene
-0.11788565376253783 countryoverparty
-0.11792356873373795 expenditure
-0.11796465590632121 taxcutsandjobsact
-0.11813268607075803 statewide
-0.11842687493469851 boulton
-0.11850884065949528 approve
-0.11857771754768619 defeated
-0.11871146233667108 presidency
-0.11874916330088689 redwave
-0.11875594757944341 ability
-0.11889142752583676 magnificent
-0.11897553053893628 greenville
-0.11903801651865129 wealthy
-0.11914846543563094 enterprise
-0.11921569910825101 patchogue
-0.11929872994975232 girl
-0.11957231008416187 beyond
-0.11974837260074203 migration
-0.11982296177173901 revenue
-0.1198486270817877 cleandreamactnow
-0.1199359151291352 enthusiasm
-0.12017076890661144 conceal
-0.12039349271904591 keep
-0.12049758771137904 phoenix
-0.12068600858295815 angry
-0.12070808861006756 exit
-0.12097540159655258 creativity
-0.12142491262056022 seopub
-0.12145428723448347 feedin

-0.3732400257343475 face
-0.37456109823353334 evening
-0.37572538212036444 click
-0.37645770627050373 plainfield
-0.37645770627050373 sludge
-0.37645770627050373 synagro
-0.3768088422543724 alabama
-0.3775180040652438 housing
-0.3785782385566046 immunization
-0.3793898721348972 nuke
-0.3818464476911152 welcome
-0.3837554318803132 safety
-0.38482352252631363 disease
-0.3866013274825667 november
-0.3868033000801038 material
-0.3876595425458134 ground
-0.38782108686699385 selling
-0.38796290991987187 infectious
-0.3881457411191333 crisis
-0.3917621606397997 discrimination
-0.3921584518128299 advocate
-0.39329547908116497 victim
-0.39341075690728894 landmark
-0.39362459400085126 please
-0.3941207564823223 plant
-0.39489055273641727 waste
-0.39556591034709726 fairfield
-0.39590314955433126 listen
-0.39636819003263757 anything
-0.3968576399940576 reuse
-0.3971520426284321 parade
-0.3978026869775303 voterobarlett
-0.3988622271599277 mobile
-0.39897438603715124 outrage
-0.3993795133138165 weal

In [377]:
c = 0
for i in y_train:
    if i == 1:
        c += 1 
print(c)
print(len(y_train))

12244
24454


In [378]:
score

0.9834437086092715

## Run Predictions

In [380]:
pred_probs = clf.predict_proba(X[len(topic_tweets) + 1:])
pred_probs[0]

array([0.99516595, 0.00483405])

In [381]:
predictions = clf.predict(X[len(topic_tweets) + 1:])
predictions

array([0, 0, 0, ..., 0, 0, 0])

In [385]:
c = 0
t = 0
tw = []
zero_and_contain_gun = 0
below_threshold = 0
tweets = full_tweets['stripped_text'].values[len(topic_tweets) + 1:]
idx = []
missed_tweets = []
tweets_above_threshold = []
for i in range(len(pred_probs)):
    if pred_probs[i][1] >= 0.99:
        t += 1
        idx.append(i)
        tweets_above_threshold.append(tweets[i])
        
    else:
        below_threshold += 1
#         if ' gun ' in tweets[i]:
#             zero_and_contain_gun += 1
#             missed_tweets.append(tweets[i])
            
# print('predicted 1 and contained gun: ', c) # predicted 1 and contained 'gun'
print("num above threshold: ", t) # total above threshold
print("below threshold: ", below_threshold)
print(zero_and_contain_gun) # predicted 0 (below threshold) and contained 'gun'

num above threshold:  819
below threshold:  529557
0


In [386]:
for t in np.random.choice(tweets_above_threshold, 100):
    print(t, '\n')

tell him this too! im sick of our babies being slaughtered while he takes copious contributions from planned parenthood under the guise of womens health 

the trump administration is telling public health doctors and scientists what words they can use when communicating with the publictelling them they cant say diversity, transgender, or fetus. thats dangerous and wrongheaded.   

the gop just passed a bill that hurts our middle and lower classes, while significantly increasing the deficit. lets invest in our people, and the health and education they need to build brighter futures for themselves and our country! #tx02 

pharmaceutical companies care more about money than people's health. it's why opioids took 42,000 lives in 2016. it's why people can't afford prescription drugs they need to survive. and it's why we need michicare single payer #medicareforall .  

i believe in an all-of-the-above approach to this issue which is why we are now addressing one of the remaining gaps, the la

In [271]:
print(len(tw))
for t in tw:
    print(t)
    print('\n')

0


In [105]:
def count_predictions(p):
    c = 0
    for p in predictions:
        if p == 1:
            c += 1
    return c

print(count_predictions(predictions))

132041
